In [1]:
from mistralai import Mistral
import json
import os
import re
import datetime
import random

In [2]:
class User:
    def __init__(self):
        pass

class ExerciseSession:
    def __init__(self, exercise_type, duration, reps, errors, created_at):
        self.exercise_type = exercise_type
        self.duration = duration
        self.reps = reps
        self.errors = errors
        self.created_at = created_at

    def to_dict(self):
        return {
            "exercise_type": self.exercise_type,
            "duration": self.duration,
            "reps": self.reps,
            "errors": self.errors,
            "created_at": self.created_at.strftime("%Y-%m-%d %H:%M:%S")
        }

# Function to generate random exercise session objects
def generate_random_sessions(num_sessions):
    exercise_types = ["push-up", "squat", "deadlift", "bench press", "pull-up"]
    sessions = []

    for _ in range(num_sessions):
        exercise_type = random.choice(exercise_types)
        duration = random.randint(20, 60)  # Duration in seconds
        reps = random.randint(10, 30)  # Number of reps
        errors = random.randint(0, 5)  # Number of errors
        created_at = datetime.datetime.now() - datetime.timedelta(days=random.randint(0, 7))  # Random date within the last week

        session = ExerciseSession(exercise_type, duration, reps, errors, created_at)
        sessions.append(session)

    return sessions

# Generate a list of random exercise session objects
random_sessions = generate_random_sessions(5)

# Convert the list of objects to a list of dictionaries
random_sessions = [session.to_dict() for session in random_sessions]

In [ ]:
class AIAssistant:
    def __init__(self, user: User):
        self.api_key = "API_KEY"
        self.model_name = "mistral-large-latest"
        self.client = Mistral(api_key=self.api_key)
        self.context = ""
        self.history = [{"role": "system", "content": "You are a fitness AI assistant. Keep your responses short and concise."}]

    def populate_context(self, data) -> None:
        """
        Formats exercise session data into structured context.
        """
        sessions = []
        for session in data:
            sessions.append({
                "exercise_type": session["exercise_type"],
                "duration": session["duration"],
                "reps": session["reps"],
                "errors": session["errors"],
                "created_at": session["created_at"],
            })

        self.context = json.dumps({
            "user_sessions": sessions
        }, indent=4)

    def generate_texts(self, data):
        self.populate_context(data)
        prompts = {
            "summary_advice": "Provide a concise summary of the user's workout performance and key takeaways.",
            "summary_analysis": "Analyze the user's workout trends, improvements, and areas needing attention.",
            "future_advice": "Give specific and actionable advice for improving future workouts based on past performance.",
            "form_score": "Provide only a number (no words) between 1-100 for the user's form score based on their workout performance.",
            "stability_score": "Provide only a number (no words) between 1-100 for the user's stability score based on their workout performance.",
            "range_of_motion_score": "Provide only a number (no words) between 1-100 for the user's range of motion score based on their workout performance."
        }

        results = {}
        for key, prompt in prompts.items():
            full_prompt = (
                f"You are a personal trainer analyzing a user's workout history. Use the session data provided to {prompt} "
                f"Here is the session data:\n"
                f"{self.context}"
            )
            self.history.append({"role": "user", "content": full_prompt})

            aiResponseObject = self.client.chat.complete(model=self.model_name,messages=self.history)
            reply = aiResponseObject.choices[0].message.content
            self.history.append({"role": "assistant", "content": reply})

            # Extract numeric values for score-related fields
            if key in ["form_score", "stability_score", "range_of_motion_score"]:
                results[key] = self.extract_numeric_value(reply)
            else:
                results[key] = reply

        return results
    
    def extract_numeric_value(self,text):
        """
        Extracts the first numeric value from a text response.
        If no number is found, it defaults to 0.
        """
        match = re.search(r"\d+", text)
        return int(match.group()) if match else 0

In [8]:
assistant = AIAssistant(User())
assistant.generate_texts(
    data=random_sessions
)

{'summary_advice': '**Summary:**\n- Consistent performance in bench press and deadlift (0 errors).\n- Improvement in pull-ups (13 to 17 reps, 0 errors in the latest session).\n- Push-ups need attention (5 errors).\n\n**Key Takeaways:**\n- Maintain form for bench press and deadlift.\n- Keep improving pull-ups.\n- Focus on push-up form to reduce errors.',
 'summary_analysis': '**Trends:** Regularly performing compound movements.\n\n**Improvements:**\n- Pull-ups: Increased reps (13 to 17) and reduced errors (2 to 0).\n- Bench press and deadlift: Consistently performing with no errors.\n\n**Areas Needing Attention:**\n- Push-ups: High error rate (5 errors in 11 reps). Needs form improvement.',
 'future_advice': '**Advice:**\n- Maintain form for bench press, deadlift, and pull-ups.\n- Focus on push-up form: Try incline push-ups or reduce speed to improve control.\n- Increase pull-up duration to match reps and reduce fatigue.\n- Consider adding variety to workouts, like squats or rows, to co